# Neuroevolución de modelos preentrenados y Word Embeddings
- ### La neuroevolución es una técnica de inteligencia artificial que combina redes neuronales con algoritmos evolutivos.
- ### Optimiza automáticamente la estructura y parámetros de una red.

## Representación del texto: Word Embeddings

- ### **Word embeddings** son representaciones numéricas densas y continuas de palabras en un espacio vectorial.
- ### Estas representaciones capturan relaciones semánticas y sintácticas entre palabras.
- ### Palabras con significados similares están más cercanas en el espacio vectorial.
- ### Densidad: Cada palabra u oración se representa como un vector en un espacio de dimensiones reducidas (por ejemplo, 100 o 300 dimensiones).

## Cargar los datos representados como word embeddings para el clasificador 

In [211]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import random
# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos


random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)
random.seed(random_state)

#cargar los datos
dataset_train = pd.read_json("./data/dataset_polaridad_es_train_embeddings.json", lines=True)
dataset_test = pd.read_json("./data/dataset_polaridad_es_test_embeddings.json", lines=True)

#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset_train.klass.value_counts())
# Extracción de los vectores de embeddings que representan a los textos: Embeddings de España 'we_es'
X_tensor_train = torch.tensor(np.vstack(dataset_train['we_es'].to_numpy()))
X_tensor_train = X_tensor_train.to(torch.float32)
# Extracción de las etiquetas o clases de entrenamiento
Y_train = dataset_train['klass'].to_numpy()

X_test = np.vstack(dataset_test['we_es'].to_numpy())
# Extracción de las etiquetas o clases de entrenamiento
Y_test = dataset_test['klass'].to_numpy()


print("Datos train:", X_tensor_train.shape) 
print("Etiquetas train:", Y_train.shape) 
print("Datos test:", X_test.shape) 
print("Etiquetas test:", Y_test.shape) 



Total de ejemplos de entrenamiento
klass
neutral     1485
positive     968
negative     689
Name: count, dtype: int64
Datos train: torch.Size([3142, 300])
Etiquetas train: (3142,)
Datos test: (786, 300)
Etiquetas test: (786,)


# Codificar las etiquetas

In [212]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_train_encoded= le.fit_transform(Y_train)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


Clases:
['negative' 'neutral' 'positive']
Clases codificadas:
[0 1 2]


In [213]:
Y_train_encoded

array([2, 1, 1, ..., 0, 1, 1], shape=(3142,))

## Definición de la arquitectura de la red para cargar el modelo preentrenado
- ### Nota. El modelo de la red debe ser identico al modelo con el que se generaron los pesos y bias en el archivo exportado 

In [214]:

# Definir la red neuronal en PyTorch heredando de la clase base de Redes Neuronales: Module
class RedNeuronal(nn.Module):
    def __init__(self, tam_entrada, tam_salida):
        super().__init__()
        # Definición de capas, funciones de activación e inicialización de pesos
        tam_entrada_capa_oculta_1 = 128
        tam_entrada_capa_oculta_2 = 8 
        self.fc1 = nn.Linear(tam_entrada, tam_entrada_capa_oculta_1)
        self.fc2 = nn.Linear(tam_entrada_capa_oculta_1, tam_entrada_capa_oculta_2)
        self.salida = nn.Linear(tam_entrada_capa_oculta_2, tam_salida)
        self.activacion= nn.PReLU()

        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.salida.weight)

        if self.fc1.bias is not None:
            nn.init.zeros_(self.fc1.bias)
        if self.fc2.bias is not None:
            nn.init.zeros_(self.fc2.bias)        
        if self.salida.bias is not None:
            nn.init.zeros_(self.salida.bias)        

    
    def forward(self, X):
        # Definición del orden de conexión de las capas y aplición de las funciones de activación
        x = self.fc1(X)
        x = self.activacion(x)
        x = self.fc2(x)
        x = self.activacion(x)
        x = self.salida(x)
        return x

# Cargar el modelo preentrenado de la red neuronal

In [215]:

# Cargar el modelo de la red neuronal desde el diccionario de pesos y sesgos
# Parámetros de la red
tam_entrada =  X_tensor_train.shape[1]
tam_salida = 3   # 3 clases

modelo_red_neuronal = RedNeuronal(tam_entrada, tam_salida)
# Se carga el modelo preentrenado
state_dict = torch.load("./data/red_neuronal_parametros.pth")
# Se cargan los pesos y bias al modelo de la red definida
modelo_red_neuronal.load_state_dict(state_dict)


<All keys matched successfully>

# Predecir datos del test con el modelo preentrenado de la red neuronal
- ### Con el objetivo de probar que funciona correctamente la red para predicción

In [216]:
from sklearn.metrics import f1_score

Y_test_encoded = le.transform(Y_test)

X_tensor_test = torch.tensor(X_test)
X_tensor_test = X_tensor_test.to(torch.float32)

# Desactivar el comportamiento de modo de  entrenamiento: por ejemplo, capas como Dropout
modelo_red_neuronal.eval()  # Establecer el modo del modelo a "evaluación"

with torch.no_grad():  # No  calcular gradientes 
    # Ejecuta el modelo: forward pass
    y_pred_test= modelo_red_neuronal(X_tensor_test)
    # Calcula la clase predica que contiene el mayor valor de la predicción
    y_pred_test = torch.argmax(y_pred_test, dim=1)
    # Calcula la métrica F1
    score = f1_score(Y_test_encoded, y_pred_test, average="macro")
    print(f"F1-score en test set: {score:.4f}")


F1-score en test set: 0.6156


# Algoritmo Genético
## Objetivo General:
- ### Evolucionar diferentes variantes de la red neuronal preentrenada (individuos) para encontrar al mejor individuo, esto es, el mejor modelo de red neuronal que maximice el rendimiento al predecir datos.


## Objetivos Particulares:
- ### 1. Generar una población de individuos que representan a la red red neuronal preentrenada con ligeras variaciones para dar diversidad de la población
- ### 2. Representar a los individuos como versiones diferentes de la red neuronal preentrenada.
- ### 3. Por medio de los operadores de variación (cruza y mutación),  evolucionar a los individuos (modelos de red neuronal) para mejorar el rendimiento al predecir nuevos datos. La evolución se realiza al cruzar y mutar los parámetros de los individuos (versiones diferentes del modelo de la red neuronal preentrenada) pesos y bias para ajustarlos mejor para la predicción de datos nuevos.

In [217]:
import torch
import random
import numpy as np
from multiprocessing import  cpu_count
from sklearn.metrics import f1_score
from joblib import Parallel, delayed


#------------------------------------------------------------------------
#------------------------------------------------------------------------
# Funciones del algoritmo genético
#------------------------------------------------------------------------
#------------------------------------------------------------------------


_TAM_ENTRADA = X_tensor_train.shape[1]
_TAM_SALIDA = 3 # 3 clases

def copiar_red_neuronal(red_neuronal, prob_global=0.8, prob_alterar=0.5, intensidad_ruido=0.01, excluir_bias=True):
    """
    Genera variantes del modelo preentrenado "red_neuronal" de manera aleatoria y con una constante de intensidad del ruido 
    para controlar el ruido por añadir a los parámetros

    red_neuronal: modelo de la red neuronal preentrenada
    prob_global: indica la probabilidad mínima para realizar una versión modificada de la red neuronal preentrenada
    prob_alterar: indica la probabilidad de modificar el parámetro (pesos y/o bias)
    intensidad_ruido: constante que modifica la intensidad de ruido que se agrega a los pesos y/o bias para generar una variante de la red neuronal preentrenada
    excluir_bias: indica si se altera el bias o no.

    """
    red_destino = RedNeuronal(_TAM_ENTRADA, _TAM_SALIDA)
    red_destino.load_state_dict(red_neuronal.state_dict())

    if random.random() < prob_global:
        # No hacer cambios, devolver copia exacta
        return red_destino
    
    with torch.no_grad():
        for name, param in red_destino.named_parameters():
            # Saltar bias si está configurado
            if excluir_bias and 'bias' in name:
                continue
                
            # Decidir si alteramos este parámetro específico 
            if random.random() < prob_alterar:
                # Devuelve un tensor con el mismo tamaño que la entrada con números aleatorios 
                # de una distribución normal con media 0 y varianza 1.
                ruido = torch.randn_like(param) * intensidad_ruido
                param.add_(ruido)        
    return red_destino

#------------------------------------------------------------------------
def inicializar_poblacion(red_neuronal, tam_poblacion):
    """
    Crea la población con versiones modificadas de la red neuronal preentrenada
    """
    poblacion = []
    for _ in range(tam_poblacion):
        poblacion.append(copiar_red_neuronal(red_neuronal))
    return poblacion

#------------------------------------------------------------------------
def funcion_fitness(red_neuronal, X, Y):
    
    """Calcula el F1-score de la red neuronal en el conjunto de datos X. Y indica la clase que se predice"""
    # Colocar la red neuronal en modo de evaluación
    red_neuronal.eval()
    with torch.no_grad():
        # Ejecuta el modelo: forward pass
        y_pred = red_neuronal(X)
        # Calcula la clase predica que contiene el mayor valor de la predicción
        y_pred = torch.argmax(y_pred, axis=1)
        # Calcula la métrica F1
        return f1_score(Y, y_pred, average="macro")  # Usar F1 macro

#------------------------------------------------------------------------
def evaluar_poblacion(poblacion, X, Y):
    """
    Evalua a todos los individuos de la población
    """
    fitness = []
    for individuo in poblacion:
        val_fitness = funcion_fitness(red_neuronal=individuo, X=X, Y=Y)
        fitness.append(val_fitness)
    return np.array(fitness)

#--------------------------------------------------------------------
def evaluar_poblacion_paralelo(poblacion,  X, Y, n_jobs=-1):
    """Evalúa la población en paralelo usando multiprocessing."""
    print(f"Evaluando población en paralelo ({n_jobs} núcleos)...")
    # Configurar el número de workers (n_jobs=-1 usa todos los núcleos)
    n_jobs = cpu_count() if n_jobs == -1 else n_jobs

    # Evaluar en paralelo

    resultados = Parallel(
    n_jobs = n_jobs-1,
    timeout = 300,  # 5min
    verbose = 10
    )(
        delayed(funcion_fitness)(red_neuronal=individuo, X=X, Y=Y)
        for individuo in poblacion
    )

    fitness = np.array(resultados)
    
    return fitness


#------------------------------------------------------------------------
def seleccionar_padres(population, fitness, k=5):
    """Selección por torneo: el mejor de dos sobrevive."""
    """Selecciona dos padres usando selección por torneo."""
    torneo = random.sample(list(zip(population, fitness)), k=k)
    # print("torneo:", torneo)
    torneo.sort(key=lambda x: x[1], reverse=True)  # Ordenar por F1-score
    return torneo[0][0], torneo[1][0]  # Retornar los dos mejores individuos

#------------------------------------------------------------------------
def elitismo(poblacion, fitness, tam_elite=2):
    """Selecciona los 'tam_elite' mejores best_individuos."""
    # Ordenar por fitness (mayor = mejor)
    ranked_indices = np.argsort(fitness)[::-1]
    elites = [poblacion[i] for i in ranked_indices[:tam_elite]]
    return elites


#------------------------------------------------------------------------
def cruzar_intercambio(padre1, padre2, prob_intercambio=0.7, prob_cruza = 0.9):
    """Cruce: intercambia pesos de dos padres de acuerdo a la probabilidad de intercambio."""
    # Si pasa el umbral de probabilidad de cruza se aplica el intercambio
    if np.random.rand() <  prob_cruza:
        # Crea el modelo de la red neuronal 
        hijo1 = RedNeuronal(_TAM_ENTRADA, _TAM_SALIDA)
        hijo2 = RedNeuronal(_TAM_ENTRADA, _TAM_SALIDA)
        with torch.no_grad():
            # padre1.parameters() Obtiene los parámetros de la red: weight y bias
            for param_h1, param_h2,  param_p1, param_p2 in zip(hijo1.parameters(), hijo2.parameters(),  padre1.parameters(), padre2.parameters()):
                mask = torch.rand_like(param_h1) > prob_intercambio
                param_h1.copy_(mask * param_p1 + (~mask) * param_p2)
                param_h2.copy_(mask * param_p2 + (~mask) * param_p1)

    else:
        hijo1, hijo2 = padre1, padre2

    return hijo1, hijo2

#------------------------------------------------------------------------
def cruzar_aritmetica(padre1, padre2, alpha=0.5, prob_cruza = 0.9):

    # Si pasa el umbral de probabilidad de cruza se aplica el intercambio
    if np.random.rand() <  prob_cruza:
        """Cruce: Aplica la cruza aritmética"""
        # Crea el modelo de la red neuronal 
        hijo1 = RedNeuronal(_TAM_ENTRADA, _TAM_SALIDA)
        hijo2 = RedNeuronal(_TAM_ENTRADA, _TAM_SALIDA)
        with torch.no_grad():
            # padre1.parameters() Obtiene los parámetros de la red: weight y bias
            for param_h1, param_h2,  param_p1, param_p2 in zip(hijo1.parameters(), hijo2.parameters(),  padre1.parameters(), padre2.parameters()):
                param_h1.copy_((alpha * param_p1) + ((1 - alpha) * param_p2))
                param_h2.copy_((alpha * param_p2) + ((1 - alpha) * param_p1))
            
    else:
        hijo1, hijo2 = padre1, padre2
    return hijo1, hijo2

#------------------------------------------------------------------------
def mutar(red_neuronal, intensidad_ruido=0.01, TASA_MUTACION = 0.1):
    """Mutación: añade ruido gaussiano a los pesos con probabilidad TASA_MUTACION e intensidad (intensidad_ruido)."""
    if np.random.rand() < TASA_MUTACION:
        with torch.no_grad():
            for param in red_neuronal.parameters():
                # Devuelve un tensor con el mismo tamaño que la entrada que se rellena con números aleatorios 
                # de una distribución normal con media 0 y varianza 1.
                ruido = torch.randn_like(param) * intensidad_ruido
                param.add_(ruido)     
    return red_neuronal

#------------------------------------------------------------------------
# --- Algoritmo Evolutivo ---
def algoritmo_evolutivo(modelo_red_neuronal, X, Y, tam_poblacion, tam_generaciones, run_paralelo=False):
    """Ejecuta el algoritmo evolutivo para optimizar la red neuronal."""
    poblacion = inicializar_poblacion(modelo_red_neuronal, tam_poblacion)
    
    for generacion in range(tam_generaciones):
        if run_paralelo:
            fitness = evaluar_poblacion_paralelo(poblacion, X, Y, n_jobs=-1)
        else:            
            fitness = evaluar_poblacion(poblacion, X, Y)        

    
        # Reporte de generación
        print(f"Generación {generacion} - Mejor precisión: {max(fitness):.4f}")
        
        nueva_poblacion = []
        for _ in range(tam_poblacion // 2):
            # Selección, cruce y mutación
            padre1, padre2 = seleccionar_padres(poblacion,fitness, k=5 )                        
            hijo1, hijo2 = cruzar_intercambio(padre1, padre2)
            nueva_poblacion.append(mutar(hijo1))
            nueva_poblacion.append(mutar(hijo2))

        #-----------------
        # Población: Los hijos sustituyen a los padres
        #-----------------
        # poblacion = nueva_poblacion
    
        #-----------------
        # Población con elitismo de padres y parte de los hijos
        #-----------------
        
        K_best_padres = 10
        poblacion[:K_best_padres] = elitismo(poblacion, fitness, K_best_padres)
        poblacion[K_best_padres:] = nueva_poblacion[K_best_padres: ]


    if run_paralelo:
        fitness = evaluar_poblacion_paralelo(poblacion, X, Y, n_jobs=-1)
    else:            
        fitness = evaluar_poblacion(poblacion, X, Y)        

    # Evaluar el mejor modelo en train
    best_individuo = poblacion[np.argmax(fitness)]
    test_f1 = funcion_fitness(best_individuo, X, Y)
    print(f"\nF1-score final en Train: {test_f1:.3f}")
    return best_individuo

## Ejecución del algoritmo evolutivo

In [218]:
best_individuo = algoritmo_evolutivo(modelo_red_neuronal, X_tensor_train, Y_train_encoded, tam_poblacion=50, tam_generaciones=50, run_paralelo=False)

Generación 0 - Mejor precisión: 0.6814
Generación 1 - Mejor precisión: 0.6881
Generación 2 - Mejor precisión: 0.6881
Generación 3 - Mejor precisión: 0.6918
Generación 4 - Mejor precisión: 0.6937
Generación 5 - Mejor precisión: 0.6937
Generación 6 - Mejor precisión: 0.6949
Generación 7 - Mejor precisión: 0.6938
Generación 8 - Mejor precisión: 0.6948
Generación 9 - Mejor precisión: 0.6970
Generación 10 - Mejor precisión: 0.6970
Generación 11 - Mejor precisión: 0.6970
Generación 12 - Mejor precisión: 0.6970
Generación 13 - Mejor precisión: 0.6970
Generación 14 - Mejor precisión: 0.6970
Generación 15 - Mejor precisión: 0.6970
Generación 16 - Mejor precisión: 0.6970
Generación 17 - Mejor precisión: 0.6970
Generación 18 - Mejor precisión: 0.6970
Generación 19 - Mejor precisión: 0.6970
Generación 20 - Mejor precisión: 0.6969
Generación 21 - Mejor precisión: 0.6969
Generación 22 - Mejor precisión: 0.6969
Generación 23 - Mejor precisión: 0.6982
Generación 24 - Mejor precisión: 0.6982
Generación

## Evaluación del mejor individuo de la evolución (mejor modelo de red neuronal evolucionada)

In [219]:

# --- Evaluación en test set ---
best_red_neuronal = best_individuo
best_red_neuronal.eval()
with torch.no_grad():
    y_pred = best_red_neuronal(X_tensor_test)
    y_pred = torch.argmax(y_pred, axis=1)
    score = f1_score(Y_test_encoded, y_pred, average="macro")

print(f"F1-score final en test set: {score:.4f}")



F1-score final en test set: 0.6342
